# Introduction

Using the development version of [scikit-learn](http://scikit-learn.org/dev/documentation.html), we construct a neural network (NN) regression model to map from a quantum mechanical potential to the energy levels (eigenvalues) for the single-particle system. The potential $V(x)$ is defined in the range $x\in [-1,1]$ and has the boundary conditions $V(\pm 1) = \infty$. The primary purpose of this notebook is to determine the optimal parameters of the NN. We do this using $10^4$ training examples. We train using these parameters. 

The potentials used for training and testing are generated in [potentials.ipynb](potentials.ipynb). The eigenvalues for all potentials are calculated in [eigenvalues.ipynb](eigenvalues.ipynb).

Documentation for `sklearn`'s supervised NN tools can be found here: http://scikit-learn.org/dev/modules/neural_networks_supervised.html. While already quite good, this documentation still appears to be "under construction".

# Preliminaries

In [14]:
%matplotlib inline
import numpy as np
from numpy.random import randint
import matplotlib.pyplot as plt
import pprint
from sklearn.model_selection import train_test_split
import pandas as pd

In [15]:
# Number of basis states for the wavefunctions
NBW = 50
nbws = np.arange(1, NBW+1)
# Number of potentials:
NV = int(1E5)
# Number of basis states in the potential:
NB = 10
ns = np.arange(1, NB+1)
# lambda (variance of Legendre coefficients):
lam = 0.75
# The variance of the n=0 legendre coefficient V_0:
V20 = 10

# Input file:
filepath = "../Data/eigenvalues_NV" + str(NV) \
    + "_NB" + str(NB) + "_lam" \
    + str(lam) + "_V20" + str(V20) + ".npy"
filepathSD = "../Data/eigenvaluesSD_NV" + str(NV) \
    + "_NB" + str(NB) + "_lam" \
    + str(lam) + "_V20" + str(V20) + ".npy"
data = np.load(filepath)
dataSD = np.load(filepathSD)
VSns = data[::,0:10]
VCns = data[::,10:20]
eigs = data[::,20::]

In [16]:
print("Data shape: ", data.shape, 
      "\nSine coefficients shape: ", VSns.shape,
      "\nCosine coefficients shape: ", VCns.shape, eigs.shape,
      "\nStd. dev. shape: ", dataSD.shape
     )

Data shape:  (100000, 60) 
Sine coefficients shape:  (100000, 10) 
Cosine coefficients shape:  (100000, 10) (100000, 40) 
Std. dev. shape:  (40,)


# Preprocessing

We know that the spectrum is symmetric under $x\to -x$. We can build this into our dataset. To do this, we duplicate the entire dataset but set all the Sine coefficients to their negative value. This is equivalent to taking $x\to-x$.

We extract the values of the potentials at a discreet, linear grid of $x$ points: $\left\{V(x) \,\, \mid \,\, x \in \{x_1,\,x_2,\ldots,x_{N_x}\}\right\}$. This grid of potential values will serve as the input to our NN model.

In [17]:
# We first define functios that help us map the Fourier-space potentials into coordinate space.
def VS(ns, xs):
    return np.sin(np.pi*np.outer(ns,xs))
def VC(ns, xs):
    return np.cos(np.pi*np.outer(ns,xs))

In [18]:
# Number of x coordinates:
Nx = 100
xs = np.linspace(-1,1,Nx)

# The coordinate space potentials:
VSs = VS(ns,xs)
VCs = VC(ns,xs)
Vgrid = np.dot(VSns,VSs) + np.dot(VCns,VCs)
VgridFlipped = np.dot(-VSns,VSs) + np.dot(VCns,VCs)

# Make sure the flip worked by looking at a random potential:
rint = randint(0, NV)
print("First 4 values of Vgrid["+str(rint)+"]:" , Vgrid[rint][0:4])
print("Last 4 values of VgridFlipped["+str(rint)+"]:", VgridFlipped[rint][-4::])

First 4 values of Vgrid[99584]: [-1.30229167 -2.15068768 -2.91993572 -3.5383973 ]
Last 4 values of VgridFlipped[99584]: [-3.5383973  -2.91993572 -2.15068768 -1.30229167]


In [19]:
numeigs = 10
X = np.concatenate( (Vgrid, VgridFlipped) )
y = np.concatenate( (eigs, eigs) )[::,1:numeigs+1]

In [20]:
# Split test and train
test_frac = 0.4
random_state = 5
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_frac, random_state=random_state
)
print("X shape, y shape: ", X.shape, y.shape)
print("X_train shape, y_train shape: ", X_train.shape, y_train.shape)
print("X_test shape, y_test shape: ", X_test.shape, y_test.shape)

X shape, y shape:  (200000, 100) (200000, 10)
X_train shape, y_train shape:  (120000, 100) (120000, 10)
X_test shape, y_test shape:  (80000, 100) (80000, 10)


# Neural network

## Pipeline
We build a pipeline with 2 steps:

1. Scale the inputs.
2. Train the Neural network.

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline

In [22]:
# The Scaler
scale = StandardScaler(with_std=False)

# The NN regression model
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
hidden_layers = (100,40)
regr =MLPRegressor(hidden_layer_sizes=hidden_layers,
                  activation = 'tanh',
                  algorithm = 'adam',
                  alpha = 0.01,
                  beta_1 = 0.85,
                  beta_2 = 0.9,
                  batch_size = 'auto',
                  max_iter = 1000,
                  tol = 1e-7,
                  learning_rate_init = 0.001,
                  verbose = True
                 )

steps = [('scale', scale), ('regr', regr)]
model = Pipeline(steps)
#pprint.pprint(model.get_params())

In [23]:
model.fit(X_train, y_train)

Iteration 1, loss = 0.01708640
Iteration 2, loss = 0.00479993
Iteration 3, loss = 0.00387746
Iteration 4, loss = 0.00349103
Iteration 5, loss = 0.00322438
Iteration 6, loss = 0.00301327
Iteration 7, loss = 0.00283696
Iteration 8, loss = 0.00267229
Iteration 9, loss = 0.00254557
Iteration 10, loss = 0.00242484
Iteration 11, loss = 0.00230843
Iteration 12, loss = 0.00220770
Iteration 13, loss = 0.00211250
Iteration 14, loss = 0.00202158
Iteration 15, loss = 0.00193451
Iteration 16, loss = 0.00185065
Iteration 17, loss = 0.00177210
Iteration 18, loss = 0.00170343
Iteration 19, loss = 0.00162687
Iteration 20, loss = 0.00155950
Iteration 21, loss = 0.00149603
Iteration 22, loss = 0.00143534
Iteration 23, loss = 0.00137889
Iteration 24, loss = 0.00132286
Iteration 25, loss = 0.00127129
Iteration 26, loss = 0.00122027
Iteration 27, loss = 0.00117450
Iteration 28, loss = 0.00113570
Iteration 29, loss = 0.00109239
Iteration 30, loss = 0.00105664
Iteration 31, loss = 0.00102028
Iteration 32, los

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=False)), ('regr', MLPRegressor(activation='tanh', algorithm='adam', alpha=0.01,
       batch_size='auto', beta_1=0.85, beta_2=0.9, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(100, 40),
       learning_rate='const...ate=None, shuffle=True, tol=1e-07, validation_fraction=0.1,
       verbose=True, warm_start=False))])

# Error

We now test the NN on the test set. We measure the error for each eigenvalue relative to the width of the distribution of that eigenvalue over all of the generated potentials. Alternatively (and similarly) one could measure the error relative to the error incurred by simply guessing that the correct value equaled the uniform square-well value.

In [24]:
y_pred = model.predict(X_test)
y_scaled_err = np.sqrt(np.mean((y_pred - y_test)**2/dataSD[0:numeigs]**2, axis = 0))
print("Scaled RMS error by eigenvalue:\n", y_scaled_err)
print("Average of scaled RMS errors:\n ", np.mean(np.abs(y_scaled_err)))

Es = ns**2 * np.pi**2 / 8.
y_rel_err = np.sqrt(np.mean((y_pred - y_test)**2/Es, axis = 0))
print("Relative RMS error by eigenvalue:\n", y_rel_err)
print("Average of relative RMS errors:\n ", np.mean(np.abs(y_rel_err)))

Scaled RMS error by eigenvalue:
 [ 0.0523901   0.06854653  0.07216696  0.09018038  0.08977839  0.11709738
  0.1448088   0.18151185  0.17096907  0.12731708]
Average of scaled RMS errors:
  0.111476652911
Relative RMS error by eigenvalue:
 [ 0.02600364  0.01124613  0.00567299  0.00362047  0.00194747  0.00144095
  0.00104226  0.00078793  0.00045897  0.00021392]
Average of relative RMS errors:
  0.00524347178507


# Timing

In [25]:
import time
start = time.time()
eigvals_pred = model.predict(Vgrid)
end = time.time()
print("Eigenvalues shape:", eigvals_pred.shape)
print("Elapsed time:", end-start, "seconds")

Eigenvalues shape: (100000, 10)
Elapsed time: 0.6253499984741211 seconds


In [27]:
60/(end-start)

95.94627032286301